# Optimize PyTorch models with Hidet
Hidet is a high-performance inference engine for deep learning models. It is designed to be easy to use and has integrated with PyTorch and ONNX frontend. In this tutorial, we will walk through how to optimize PyTorch models with Hidet.

To run the tutorial, you need to install PyTorch 2.0, Hidet and jupyter notebook:
```bash
$ pip install --upgrade torch torchaudio torchvision
$ pip install hidet jupyter
```

Then, you can launch the jupyter notebook by running `jupyter notebook`
```bash
$ jupyter notebook
```

and open this tutorial in the browser.

In [1]:
import torch

assert torch.__version__ >= '2.0'

The most convenient way to use Hidet is to use the `torch.compile` API. It will automatically optimize the model by dispatching it to the specified backend. This feature is introduced in PyTorch 2.0.


In [2]:
model = torch.hub.load(
    'pytorch/vision:v0.6.0',
    model='resnet18',
    pretrained=True,
    verbose=False
).cuda().eval()
x = torch.randn(1, 3, 224, 224).cuda()
y1 = model(x)

We will use the ResNet 50 model as an example. The model is downloaded from the PyTorch model zoo. The model is first run on the CPU to get the reference output `y1`.


In [3]:
import hidet
# hidet.torch.dynamo_config.search_space(2)
model_opt = torch.compile(model, backend='hidet')
y2 = model_opt(x)

We can optimize a model by calling `torch.compile` with the `backend` argument set to `hidet`. The optimized model is returned as `model_opt`. The optimized model is run on the GPU to get the output `y2`.

Hidet will automatically conduct a series of graph-level optimizations and generate kernels for each operator. We can search the most performant kernel for each operator and its input shape by setting the search space. By default, the search space is 0 and indicates no searching. The search space can be set to 1 or 2. The larger the search space is, the more kernels will be searched and the longer the compilation time will be. The search space can be set by calling `hidet.torch.dynamo_config.search_space(1 or 2)`.


In [4]:
# Check the results
torch.testing.assert_close(actual=y2, expected=y1, rtol=1e-2, atol=1e-2)

The results are compared to the reference output `y1` to make sure the optimization is correct.

### Hidet options
Besides the `search_space`, hidet provides several options to control the optimization. All the options can be set by calling `hidet.torch.dynamo_config.xxx()` where `xxx` is the option name. The following options are available:

In [5]:
config = hidet.torch.dynamo_config
config.search_space(level=2)  # set the search space, level can be 0, 1 or 2.
config.correctness_report()  # print the correctness report (the error between the optimized model and the reference pytorch model)
config.use_fp16()  # convert the model to FP16
config.dump_graph_ir(output_dir='./outs')  # debug: dump the hidet graph IR
config.print_input_graph()  # debug: print the received torch.fx.Graph from torch dynamo